Transformer架构

阶段                 输入维度              输出维度
嵌入层(Embedding)     (B,L)               (B,L,D)

位置编码              (B,L,D)              (B,L,D)

中间运算层             (B,L,D)              (B,L,D)

分类层(Linear)        (B,L,D)              (B,L,V)

每个板块的核心本质：
- 嵌入层（Embedding）：特征映射将离散的单词ID映射为连续的向量表示，为模型提供初始的语义坐标（连续向量可以用于做向量运算）。
- 位置编码（Positional Encoding）：通过注入位置信息，让模型知道序列中每个元素的相对位置，打破词袋模型的局限性。
- 中间运算层（Self-Attention & MLP）：通过注意力机制，让模型能够关注序列中的不同位置，动态调整特征表示。
- 分类层（Linear）：回归预测将深层特征映射回庞大的词表空间，计算下一个词的概率分布。


一、基本板块的构建

In [ ]:
#  1. 自定义线性层的实现（没有偏置项）
import torch
import torch.nn as nn
import torch.nn.init as init
import math

class Linear(nn.Module):
    def __init__(self, in_features, out_features, device=None, dtype=None):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features

        factory_kwargs = {'device':device, 'dtype':dtype}
        self.weight = nn.Parameter(torch.empty((out_features, in_features), **factory_kwargs))

        # 截断正态分布作为初始化权重（可以自定义）
        self.reset_parameters()
    
    def reset_parameters(self):
        # 计算标准差sigma
        # sigma^2 = 2/(d_in + d_out)
        sigma = math.sqrt(2.0 / (self.in_features + self.out_features))
        
        # 自定义截断范围[-3sigma, 3sigma]
        # trunc_normal_的参数：tensor, mean, std, a(min), b (max)
        init.trunc_normal_(
            self.weight,
            mean = 0.0,
            std = sigma,
            a = -3.0*sigma,
            b = 3.0*sigma
        )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # 权重为(out, in)，输入文本数据为(batch, in)其中batch表示处理多少样本in表示特征向量也就是每句话的编码长度 
        # 需要用.t()对权重进行转置
        return x @ self.weight.t()

In [ ]:
# 测试自定义线性层
model = Linear(out_features=3, in_features=6)
test_weights = torch.randn(3, 6)
state_dict = {'weight': test_weights}
model.load_state_dict(state_dict)

x = torch.randn(1,6)
output = model(x)
print(f"权重为：{model.weight}")
print(f"输入为：{x}")
print(f"输出为：{output}")

manual = x @ test_weights.t()
print(f"验证结果是否一致：{torch.allclose(output, manual)}")

In [ ]:
# 2.自定义嵌入模块（Embedding）
import torch
import torch.nn as nn
import torch.nn.init as init

class Embedding(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, device=None, dtype=None):
        super().__init__()

        # 权重矩阵(num_embeddings, embedding_dim)->(词汇表大小，嵌入向量维度)
        factory_kwargs = {'device': device, 'dtype': dtype}
        self.weights = nn.Parameter(torch.empty((num_embeddings, embedding_dim), **factory_kwargs))

        # 使用正态函数截断初始化权重
        self.reset_parameters()

    def reset_parameters(self):
        sigma = 1.0
        init.trunc_normal_(
            self.weights,
            mean = 0.0,
            std = sigma,
            a = -3.0*sigma,
            b = 3.0*sigma
        )   
    def forward(self, token_ids: torch.Tensor) -> torch.Tensor:
        # 输入token_ids为（处理批次大小，序列长度）
        # 输出为（处理批次大小，序列长度，嵌入向量维度）
        return self.weights[token_ids]


In [ ]:
# 测试Embedding模块
import random
# run_embedding函数相当于映射token_ids到weights矩阵的行向量
def run_embedding(num_embeddings, embedding_dim, weights,token_ids):
    # 测试Embedding模块
    model = Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim)

    state_dict = {'weights': weights}
    model.load_state_dict(state_dict)

    model.eval()
    with torch.no_grad():
        output = model(token_ids)
    return output

weights = torch.randn(10, 3)
# 测试框架提供的输入(batch, seq_len),这里模拟输入两句话每句话有3个token
ids = torch.tensor([[2, 9, 5], [3, 2, 6]])
f = run_embedding(10, 3, weights, ids)
print(f"输出embedding向量: {f}")

print(f"权重矩阵: {weights}")

check_1 = torch.equal(f[0][0], weights[2])
check_2 = torch.equal(f[1][2], weights[6])

print(f"Index 2匹配: {check_1}")
print(f"Index 6匹配: {check_2}")

Q: 反向传播中更新改变了什么？

A: 在反向传播中，更新权重改变的是单词的‘内涵’，而不是‘名字’:
    1. 不变关系：词表里的token ID（比如weights[2]对应猫）是由预分词阶段决定的，在Transformer训练时不会改动这个索引关系。
    2. 数值变换：权重矩阵的每一行就像是一张token坐标记录表，transformer反向传播修改的是表里那些浮点数，改变这些数值本质上是在改变这个token在语义空间里的精确定位，通过不断微调让含义相近的词在地图上靠得更近，从而让模型“理解”语言（比如‘猫’和‘狗’在语义空间里的距离更近即向量空间夹角更小，说明它们的含义更接近）。

In [ ]:
# 3. 实现RMSNorm归一化层
# RMSNorm的核心思想是对输入特征进行缩放而不是平移
import torch
import torch.nn as nn

class RMSNorm(nn.Module):
    def __init__(self, d_model: int, eps=1e-5, device=None, dtype=None):
        super().__init__()
        self.eps = eps

        # 归一层可学习的“增益”参数，初始化为1
        self.weight = nn.Parameter(torch.ones(d_model, device=device, dtype=dtype))

    def forward(self,x: torch.Tensor) -> torch.Tensor:
        # 输入x的形状为（处理批次大小，序列长度，嵌入向量维度）
        origin_dtype = x.dtype

        # x转化为float32保证归一化的过程没有数值下溢
        x_fp32 = x.to(torch.float32)

        # mean(-1, keepdim=True)表示除以最后一维数值即嵌入向量维度d_model
        # torch.rsqrt(x)表示1/torch.sqrt(x)
        RMS = torch.rsqrt(x_fp32.pow(2).mean(-1, keepdim=True)+self.eps)
        x_normed = x_fp32 * RMS * self.weight.to(torch.float32)
        return x_normed.to(origin_dtype)

In [ ]:
# 测试归一化层
import random
def run_rmsnorm(d_model: int, x: torch.Tensor, eps: float):
    # 实例化你刚刚写好的类
    norm_layer = RMSNorm(d_model=d_model, eps=eps, device=x.device, dtype=x.dtype)
    return norm_layer(x)

x = torch.randn(2,3)
f = run_rmsnorm(3, x, 1e-5)
print(f"输入为：{x}")
print(f"归一化输出为：{f}")

Q: RMSNorm的作用和优势是什么？

A: 
* 在深度学习中，归一化的核心差异在于“统计的维度”。LLM选择层归一化（LayerNorm）而不是批归一化（BatchNorm）：

    1. BatchNorm(批归一化)：统计跨样本的同一特征。LLM中，由于序列长度波动大、显存限制导致Batch Size较小，且不同样本间的统计特性差异巨大，BatchNorm会破坏特征的一致性。
    2. LayerNorm、RMSNorm(层归一化)：统计同一样本内的不同特征。
    3. 语义完整性：LLM的语义信息编码在每个Token的向量空间分布中。层归一化在单个样本内部进行统计，能够更好地捕捉和保留特定上下文（Context）下的语义特征，而不受其他样本干扰。

* RMSNorm vs. LayerNorm：从“全处理”到“精简化”
LayerNorm包含两个操作：平移（Re-centering）和缩放（Re-scaling）
RMSNorm做了减法：它假设均值偏移对模型性能贡献微小，因此取消了均值减法，仅保留缩放操作。

* RMSNorm的核心优势
A. 计算效率的飞跃
    1. 逻辑：减少了求均值、减均值的两次全局遍历和减法运算。
    2. 效果： 在处理超大规模参数（如70B+）时，累积减少的计算开销能显著提升模型训练与推理的吞吐量（Throughput）。

B. 更加温和的残差分支保护
    1. 逻辑：现代LLM普遍采用Pre-Norm架构（归一化在残差连接之前）。
    2. 深度理解：LayerNorm强制将每一层数据的分布中心平移到0，而RMSNorm不改变数据的中心，它仅对向量的幅值进行规范化。
    3. 优势：这种“不强制改变分布中心”的操作更温和地保留了原始残差分支的信息。在超深网络中，这有助于维持梯度的健康流动，防止模型在深层出现特征退化。

C. 保持特征表示强度
    1. 逻辑：RMSNorm通过控制向量的“模长”（Magnitude）来防止激活值爆炸。
    2. 优势：它在保证数值稳定的同时，保留了各特征维度之间的相对比例关系，从而确保了模型对Token特征表示的强度。

💡定义：RMSNorm是一种计算成本更低、更契合大规模生成式任务的归一化方案。它通过舍弃LayerNorm的平移不变性，换取了更高的计算性能和对深层残差信息的尊重。这是大模型在性能（Speed）与稳定性（Stability）之间取得的卓越平衡。



In [ ]:
# 4. 实现SwiGLU激活函数 <- SiLU+门控机制GLU
import torch
import torch.nn as nn

class SwiGLU(nn.Module):
    def __init__(self, d_model: int):
        super().__init__()

        # 计算隐藏层维度d_ff，设为8/3 * d_model，并向上取整到64的倍数
        d_ff = int(8 / 3 * d_model)
        d_ff = (d_ff + 63) // 64 * 64
        
        # 定义线性层，Gate投影（W）和Up（V）投影往往合并或并行定义
        # 输入(in_features)即词嵌入向量维度，输出(out_features)即隐藏层维度d_ff
        self.w_gate = Linear(d_model, d_ff)
        self.w_up = Linear(d_model, d_ff)
        # Down投影将维度转回d_model
        self.w_down = Linear(d_ff, d_model)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # 建议使用torch.sigmoid来实现SiLU，即SiLU(x)=x*sigmoid(x)
        gate = self.w_gate(x)
        swish_gate = gate * torch.sigmoid(gate)
        intermediate = swish_gate * self.w_up(x)
        
        # 最后投影回原始维度
        return self.w_down(intermediate)


In [ ]:
# 测试SwiGLU
def run_swiglu(d_model: int, x: torch.Tensor):
    """
    测试适配器：实例化SwiGLU并运行前向传播
    """
    model = SwiGLU(d_model).to(x.device).to(x.dtype)
    model.eval()
    with torch.no_grad():
        output = model(x)
    return output   
d_model = 3
x = torch.randn(1, 5, d_model)
output = run_swiglu(d_model, x)
print(f"输入形状: {x.shape}")
print(f"输出形状: {output.shape}")
print(f"输入：{x}")
print(f"输出：{output}")


In [ ]:
# 5. 实现RoPE位置编码 <- 旋转位置编码（相对位置编码对长上下文友好）
import torch
import torch.nn as nn

class RotaryPositionalEmbedding(nn.Module):
    def __init__(self, theta: float, d_k: int, device=None):
        super().__init__()
        self.d_k = d_k
        self.theta = theta
    
        # 初始化为空，方便forward动态生成（预计算并缓存cos和sin值）
        self.register_buffer("cos", None, persistent=False)
        self.register_buffer("sin", None, persistent=False)

    def _build_cache(self, seq_len, device, dtype):
        # 只有在seq_len超过当前缓存时才重新计算
        if self.cos is not None and seq_len <= self.cos.shape[0]:
            return 
        powers = torch.arange(0, self.d_k, 2, device=device).float()
        inv_freq = self.theta ** (-powers / self.d_k)

        t = torch.arange(seq_len, device=device).float()
        freqs = torch.outer(t, inv_freq)  # freqs.shape:(seq_len, d_k/2)

        # 缓存cos和sin值
        self.register_buffer("cos", torch.cos(freqs).to(dtype), persistent=False)
        self.register_buffer("sin", torch.sin(freqs).to(dtype), persistent=False)

    def forward(self, x: torch.tensor) -> torch.Tensor:
        # x.shape: (batch_size, num_heads, seq_len, d_k)
        seq_len = x.shape[-2]

        # 动态判定并生成缓存
        self._build_cache(seq_len, x.device, x.dtype)

        # 切片获取当前长度所需的缓存
        # d_model/num_heads也必须为偶数
        assert self.d_k % 2 == 0, "RoPE需要d_k是一个偶数"
        cos = self.cos[:seq_len, :] # (seq_len, d_k/2)
        sin = self.sin[:seq_len, :] # (seq_len, d_k/2)

        # 实现旋转逻辑并把d_k维拆分为两半处理
        # cos、sin形状从(S, D/2)->(1, 1, S, D/2)使用unsqueeze适配多头注意力机制维度广播
        cos = cos.view(1, seq_len, -1)
        sin = sin.view(1, seq_len, -1)
        
        x1 = x[..., 0::2]
        x2 = x[..., 1::2]
        
        # 旋转矩阵公式：[x1*cos - x2*sin, x1*sin + x2*cos] 
        # x_rot.shape:(B, H, S, D/2, 2)
        x_rot = torch.stack([x1 * cos - x2 * sin, x1 * sin + x2 * cos], dim=-1)

        return x_rot.flatten(-2) # 最后展平回到(B, H, S, D)

In [ ]:
# 测试RoPE
import random
def run_rope(theta: float, d_k: int, x: torch.Tensor) -> torch.Tensor:
    # 实例化并运行RoPE
    model = RotaryPositionalEmbedding(theta, d_k, device=x.device)
    return model(x)

x = torch.randn(1, 2, 5, 8)
print(f"输入：{x}")
print(f"输出：{run_rope(10000.0, 8, x)}")

Q: 旋转位置编码的优势在于什么，以及theta如何选择？

A: 旋转位置编码（RoPE）的核心优势在于：它通过对Attention中的Query和Key向量施加与位置相关的旋转，将相对位置信息直接融入注意力计算过程。该旋转规则在整个训练与推理过程中是固定的，因此对应的cos、sin值可以预先计算并以buffer的形式缓存和复用。从本质上看，RoPE为Attention引入了一种几何约束框架，使得注意力分数显式依赖于token之间的相对位置差，而非绝对位置。由于二维旋转矩阵的点积性质，经过旋转后的等价于在点积中引入与𝑗−𝑖相关的相位偏移，从而使模型能够自然地建模相对距离关系，而无需额外的相对位置偏置或可学习参数。这一特性使得RoPE在长上下文外推以及推理阶段（如使用KV cache）具有良好的稳定性和泛化能力。

其中，theta是控制旋转频率分布的超参数：theta越大，整体旋转频率越低，更有利于建模长距离（全局）依赖；theta越小，高频成分占比越大，更强调局部相对位置关系。在实践中，theta = 10000.0 是被广泛验证的默认选择，它在同一表示空间中同时覆盖了局部与全局相对位置建模需求，因此被大多数主流大语言模型采用。

In [ ]:
# 6. 实现softmax函数
def Softmax(x: torch.Tensor, dim:int) -> torch.Tensor:
    # 减去最大值提高稳定性,取指定维度dim的最大值,保持取出维度一致
    max_val = torch.max(x, dim=dim, keepdim=True)[0]
    e_x = torch.exp(x - max_val)
    return e_x / e_x.sum(dim=dim, keepdim=True)   # 所有数值都在[0,1)之间,且和为1


In [ ]:
# 测试softmax函数
def run_softmax(x: torch.Tensor, dim: int):
    return softmax(x, dim)
test_input = torch.randn(2, 3)
output = Softmax(test_input, dim=-1)
print(f"输入: {test_input}")
print(f"输出: {output}")


In [ ]:
# 7. 实现缩放点积注意力机制
import math

def scaled_dot_product_attention(q: torch.Tensor, k: torch.Tensor, v: torch.Tensor, mask= None):
    """
    缩放点积注意力机制
    Args:
        q: 查询,形状为(batch_size, num_heads, seq_len, d_k)
        k: 键,形状为(batch_size, num_heads, seq_len, d_k)
        v: 值,形状为(batch_size, num_heads, seq_len, d_v)
        mask: 掩码形状为(seq_len, seq_len)布尔张量, True表示保留该位置的注意力权重，False表示屏蔽该位置的注意力权重
    """
    d_k = q.size(-1)
    scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)

    # 应用掩码mask
    if mask is not None:
        scores = scores.masked_fill(mask == False, float('-inf'))
    attn_weights = Softmax(scores, dim=-1)

    # 得分权重乘以v得到最后加权结果输出
    output = torch.matmul(attn_weights, v)

    return output, attn_weights
    

In [ ]:
# 测试点积函数的实现
import torch
import math

def test_implementation():
    # 设置随机种子保证结果可复现
    torch.manual_seed(42)
    
    # 维度定义
    batch_size = 2
    heads = 1
    seq_len = 3
    d_k = 5
    d_v = 5

    # 3阶张量输入(batch_size, seq_len, d_k)
    q3 = torch.randn(batch_size, seq_len, d_k)
    k3 = torch.randn(batch_size, seq_len, d_k)
    v3 = torch.randn(batch_size, seq_len, d_v)
    
    out3, attn_weights = scaled_dot_product_attention(q3, k3, v3)
    print(f"3阶张量输出:\n{out3}")
    assert out3.shape == (batch_size, seq_len, d_v), f"3D形状错误: {out3.shape}"
    print("3阶张量形状测试通过")

    # 4阶张量输入(batch_size, heads, seq_len, d_k)
    q4 = torch.randn(batch_size, heads, seq_len, d_k)
    k4 = torch.randn(batch_size, heads, seq_len, d_k)
    v4 = torch.randn(batch_size, heads, seq_len, d_v)
    
    out4, attn_weights = scaled_dot_product_attention(q4, k4, v4)
    print(f"4阶张量输出:\n{out4}")
    assert out4.shape == (batch_size, heads, seq_len, d_v), f"4D形状错误: {out4.shape}"
    print("4阶张量形状测试通过")

    # 掩码测试
    mask = torch.tril(torch.ones(seq_len, seq_len)).bool()
    print(f"掩码矩阵mask:\n{mask}")

    # 如果掩码生效，第一行输出应该只与第一个Value有关
    out_masked, attn_weights = scaled_dot_product_attention(q3, k3, v3, mask=mask)
    print(f"注意力权重attn_weights:\n{attn_weights}")
    
    # 验证掩码逻辑：在计算注意力权重时，我们可以通过检查输出是否包含NaN或Inf来初步判断
    print(f"掩码后的输出out_masked:\n{out_masked}")
    assert not torch.isnan(out_masked).any(), "输出包含NaN"
    print("掩码功能测试通过")


# 运行测试
if __name__ == "__main__":
    try:
        test_implementation()
    except NameError:
        print("错误：请先定义scaled_dot_product_attention函数再运行测试。")
    except Exception as e:
        print(f"测试失败{e}")

In [ ]:
# 8. 因果多头注意力机制实现
import torch
import torch.nn as nn
import math

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, num_heads: int):
        super().__init__()
        assert d_model % num_heads == 0, "d_model必须能被num_heads整除"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.w_q = Linear(d_model, d_model)
        self.w_k = Linear(d_model, d_model)
        self.w_v = Linear(d_model, d_model)
        self.w_o = Linear(d_model, d_model)

        # 对Q、k使用RoPE
        self.rope = RotaryPositionalEmbedding(10000.0, d_k=self.d_k)

    def forward(self, x, mask=None):
        # x.shape->(batch_size, seq_len, d_model)
        batch_size, seq_len, _ = x.shape

        # 线性变换并拆分为多头 (B, L, D) -> (B, S, H, D_K) -> (B, H, S, D_K)
        q = self.w_q(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        k = self.w_k(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        v = self.w_v(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)

        q = self.rope(q)
        k = self.rope(k)
        
        # 计算缩放点积得分
        scores, _ = scaled_dot_product_attention(q, k, v, mask=mask)
        
        # 加权求和并拼接头 (B, H, S, D_K)->(B, S, H, D_K)->(B, S, D),其中D=D_K*H=d_model
        out = scores.transpose(1, 2).contiguous()
        out = out.view(batch_size, seq_len, self.d_model)

        return self.w_o(out)

In [ ]:
# 测试多头注意力机制
import torch

def run_multihead_self_attention(d_model, num_heads, x):
    """
    测试适配器,x:输入张量(batch_size, seq_len, d_model)
    """
    model = MultiHeadAttention(d_model, num_heads)
    model.eval()
    return model(x)

x = torch.randn(1, 2048, 8)
out = run_multihead_self_attention(8, 2, x)
print(f"输出形状: {out.shape}")
print(f"输出: {out}")

In [ ]:
# 9. 实现Transformer
import torch
import torch.nn as nn

class TransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        # 参数：
        # d_model：输入维度（词向量嵌入维度）
        # num_heads：头数
        # d_ff：前馈神经网络维度
        super().__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)

        self.ffn = nn.Sequential(
            Linear(d_model, d_ff),
            SwiGLU(d_ff),
            Linear(d_ff, d_model)
        )
        
        self.norm1 = RMSNorm(d_model)
        self.norm2 = RMSNorm(d_model)

    def forward(self, x: torch.Tensor, mask=None):
        original_x1 = x

        # Pre-Norm + Multihead Attention + Residual Connection
        x = self.norm1(x)
        x = self.attention(x)
        x = x + original_x1
        
        # Pre-Norm + FFN + Residual Connection
        original_x2 = x
        x = self.norm2(x)
        x = self.ffn(x)
        x = x + original_x2
        return x

In [ ]:
# 测试TransformerBlock
def run_transformerBlock(d_model, num_heads, d_ff):
    model = TransformerBlock(d_model=d_model, num_heads=num_heads, d_ff=d_ff)
    model.eval()
    return model(x)

# 测试参数
d_model = 4
num_heads = 2
d_ff = 2

# 生成随机输入
x = torch.randn(1, 5, d_model)

# 运行模型
output = run_transformerBlock(d_model, num_heads, d_ff)

print(f"输入形状: {x.shape}")
print(f"输入示例: {x}")

print(f"输出形状: {output.shape}")
print(f"输出: {output}")


二、 支持Transformer训练的板块构建

In [ ]:
# 1. 实现交叉熵损失函数
import numpy as np

def cross_entropy(logits, targets):
    # logits:模型输出的 logits，形状为(batch_size, Vocab)
    # targets: 真实标签，形状为(batch_size,)

    # 为了保持数值稳定性，我们从每个样本的logits中减去最大值
    max_val = np.max(logits, axis=-1, keepdims=True)
    shifted_logits = logits - max_val

    log_sum_exp = np.log(
        np.sum(np.exp(shifted_logits), axis=-1, keepdims=False)
    )

    target_logits = np.take_along_axis(shifted_logits, targets[..., None], axis=-1).squeeze(-1)
    loss_i = log_sum_exp - target_logits
    loss = np.mean(loss_i)

    return loss

def calculate_perplexity(logits, targets):
    # 用交叉熵损失函数计算困惑度
    loss = cross_entropy(logits, targets)
    
    # 困惑度 = exp(平均损失)
    perplexity = np.exp(loss)

    return perplexity


In [ ]:
# 2. 自定义实现AdamW优化器
import torch
from torch.optim import Optimizer

class AdamW(Optimizer):
    def __init__(
        self,
        params,
        lr=1e-3,
        betas=(0.9, 0.999),
        eps=1e-8,
        weight_decay=0.0,
    ):
        """
        参数:
            params: 待优化的参数迭代器(可学习参数)
            lr: 学习率
            betas: 用于计算梯度及其平方的运行平均值的系数(一阶矩, 二阶矩)
            eps: 添加到分母以提高数值稳定性
            weight_decay: 权重衰减系数 (L2正则化解耦)
        """
        # 参数校验
        if lr <= 0.0:
            raise ValueError(f"Invalid learning rate: {lr}")
        if eps <= 0.0:
            raise ValueError(f"Invalid epsilon value: {eps}")
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError(f"Invalid beta1: {betas[0]}")
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError(f"Invalid beta2: {betas[1]}")

        # 将超参数打包进dict，方便父类Optimizer管理
        defaults = dict(
            lr=lr,
            betas=betas,
            eps=eps,
            weight_decay=weight_decay,
        )
        super().__init__(params, defaults)

    @torch.no_grad()
    def step(self, closure=None):
        """
        执行一步优化更新
        """
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        # 遍历所有的参数组（例如模型不同层可以有不同的学习率）
        for group in self.param_groups:
            lr = group["lr"]
            beta1, beta2 = group["betas"]
            eps = group["eps"]
            weight_decay = group["weight_decay"]

            for param in group["params"]:
                # 如果参数没有梯度，则跳过
                if param.grad is None:
                    continue

                grad = param.grad
                state = self.state[param]

                # 初始化状态变量（只在第一次迭代时执行）
                if len(state) == 0:
                    state["step"] = 0
                    # 指数移动平均：一阶矩（动量）
                    state["exp_avg"] = torch.zeros_like(param)
                    # 指数移动平均：二阶矩（梯度的平方）
                    state["exp_avg_sq"] = torch.zeros_like(param)

                exp_avg = state["exp_avg"]
                exp_avg_sq = state["exp_avg_sq"]

                state["step"] += 1
                t = state["step"]

                # 更新一阶矩：m_t = β1 * m_{t-1} + (1 - β1) * g_t
                # mul_ 是原地乘法，add_ 是原地加法
                exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)

                # 更新二阶矩：v_t = β2 * v_{t-1} + (1 - β2) * g_t^2
                # addcmul_计算：exp_avg_sq = exp_avg_sq + (1 - beta2) * (grad * grad)
                exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1 - beta2)

                # 计算偏置修正
                # 由于m和v初始化为0，早期迭代会向0偏移，需要修正
                bias_correction1 = 1 - beta1 ** t
                bias_correction2 = 1 - beta2 ** t

                # 计算最终的步长：修正后的学习率
                step_size = lr * (bias_correction2 ** 0.5) / bias_correction1

                # 更新参数：θ = θ − step_size * m / (sqrt(v) + ε)
                denom = exp_avg_sq.sqrt().add_(eps)
                param.addcdiv_(exp_avg, denom, value=-step_size)

                # 解耦权重衰减
                # θ = θ - lr * weight_decay * θ
                if weight_decay != 0:
                    param.add_(param, alpha=-lr * weight_decay)
        return loss

In [ ]:
# 3. 实现LLamA类似的余弦退火学习率调度器
import math

def get_lr_cosine_schedule(t, alpha_max, alpha_min, T_w, T_c):
    """
    参数:
        t: 当前迭代步数
        alpha_max: 最大学习率
        alpha_min: 最小（最终）学习率
        T_w: 预热迭代次数
        T_c: 余弦退火迭代总次数
    """
    
    if t < T_w:
        return (t / T_w) * alpha_max
    
    elif T_w <= t <= T_c:
        progress = (t - T_w) / (T_c - T_w)
        cosine_out = 0.5 * (1 + math.cos(math.pi * progress))
        return alpha_min + cosine_out * (alpha_max - alpha_min)
    else:
        return alpha_min

In [ ]:
# 测试余弦退火学习调度器的代码
import math
import matplotlib.pyplot as plt 

# 模拟测试
A_MAX = 1e-3
A_MIN = 1e-5
TW = 20    # 第20步完成预热
TC = 100   # 100步完成退火
TOTAL_STEPS = 120

steps = list(range(TOTAL_STEPS))
lrs = [get_lr_cosine_schedule(t, A_MAX, A_MIN, TW, TC) for t in steps]

# 打印关键节点数值
print(f"Step 0(开始): {lrs[0]:.6f}")
print(f"Step {TW}(预热结束): {lrs[TW]:.6f} (预期应接近{A_MAX})")
print(f"Step {(TW+TC)//2}(退火中点): {lrs[(TW+TC)//2]:.6f}")
print(f"Step {TC}(退火结束): {lrs[TC]:.6f} (预期应为{A_MIN})")
print(f"Step {TOTAL_STEPS-1}(退火后): {lrs[-1]:.6f} (预期应保持{A_MIN})")

plt.plot(steps, lrs)
plt.axvline(x=TW, color='r', linestyle='--', label='Warm-up End')
plt.axvline(x=TC, color='g', linestyle='--', label='Annealing End')
plt.legend()
plt.show()

In [ ]:
# 4. 梯度裁剪
import torch

def run_gradient_clipping(params, max_norm, eps=1e-6):
    """
    参数:
        params: 包含待裁剪梯度的参数列表或生成器
        max_norm: 允许的最大L2范数(M)
        eps: 用于数值稳定性的微小值
    """
    params_with_grad = [p for p in params if p.grad is not None]
    if len(params_with_grad) == 0:
        return

    total_norm = torch.norm(
        torch.stack([torch.norm(p.grad.detach(), 2) for p in params_with_grad]), 2
    )

    clip_coeff = max_norm / (total_norm + eps)

    if clip_coeff < 1.0:
        for p in params_with_grad:
            p.grad.detach().mul_(clip_coeff)

In [ ]:
# 测试梯度裁剪代码
import torch

def test_gradient_clipping_logic():
    print("开始梯度裁剪模拟测试...")
    
    # 创建两个参数，手动设置梯度
    p1 = torch.tensor([1.0, 2.0], requires_grad=True)
    p2 = torch.tensor([2.0, 2.0], requires_grad=True)
    
    # 手动赋予较大的梯度：g1=[3.0, 0.0], g2=[0.0, 4.0]
    p1.grad = torch.tensor([3.0, 0.0])
    p2.grad = torch.tensor([0.0, 4.0])
    
    # 计算当前全局L2范数：sqrt(3^2 + 4^2) = 5.0
    max_norm = 1.0  # 设置最大范数为1.0
    print(f"\n场景 1(触发裁剪):")
    print(f"裁剪前 - 原始梯度范数: 5.0, 目标最大范数M: {max_norm}")
    
    run_gradient_clipping([p1, p2], max_norm)
    
    # 裁剪后的总范数应接近1.0(由于eps的存在，会略小于1.0)
    new_norm = torch.norm(torch.stack([torch.norm(p1.grad, 2), torch.norm(p2.grad, 2)]), 2)
    print(f"裁剪后 - 参数1 梯度: {p1.grad}")
    print(f"裁剪后 - 参数2 梯度: {p2.grad}")
    print(f"裁剪后 - 全局梯度范数: {new_norm.item():.6f} (预期<= {max_norm})")

    # 梯度小于最大范数 (不应裁剪)
    p3 = torch.tensor([0.1, 0.1], requires_grad=True)
    p3.grad = torch.tensor([0.1, 0.2])
    
    # 当前范数约为0.2236
    max_norm_large = 10.0
    original_grad = p3.grad.clone()
    
    print(f"\n场景 2(不触发裁剪):")
    print(f"裁剪前 - 原始梯度范数: 0.2236, 目标最大范数M: {max_norm_large}")
    
    run_gradient_clipping([p3], max_norm_large)
    
    print(f"裁剪后 - 梯度是否保持不变: {torch.equal(p3.grad, original_grad)}")
    print(f"裁剪后 - 梯度范数: {torch.norm(p3.grad, 2).item():.6f}")

if __name__ == "__main__":
    test_gradient_clipping_logic()

In [ ]:
# 5. 数据加载（给定前面的token预测下一个token）
import torch
import numpy as np
import random
def get_batch(x, batch_size, context_length, device):
    """
    x: numpy数组(token ID)
    batch_size: 批大小
    context_length: 上下文长度(m)
    device: 设备字符串 ('cpu', 'cuda', 'mps')
    """

    # 确保i + context_length之后还有一个位置留给目标序列的最后一个令牌
    max_idx = len(x) - context_length
    
    # 随机生成batch_size个起始位置
    ix = torch.randint(0, max_idx, (batch_size,))
    
    # 根据索引提取输入和目标
    # x_batch: 形状为(batch_size, context_length)
    # y_batch: 形状为(batch_size, context_length)
    x_batch = torch.stack([torch.from_numpy(x[i : i + context_length].astype(np.int64)) for i in ix])
    y_batch = torch.stack([torch.from_numpy(x[i + 1 : i + context_length + 1].astype(np.int64)) for i in ix])
    
    # 将张量移动到指定设备
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)
    
    return x_batch, y_batch

In [ ]:
# 测试数据加载函数
import numpy as np
import torch
def run_get_batch(x_list, batch_size, context_length, device_str):
    """
    测试框架可能会以列表形式传入数据，需要确保它是numpy数组
    """
    # 将输入转换为numpy数组（如果它还不是的话）
    x = np.array(x_list)

    # 调用实现的get_batch函数
    x_batch, y_batch = get_batch(x, batch_size, context_length, device_str)
    
    return x_batch, y_batch

x = np.array([1, 2, 3, 2, 5, 9, 10, 11, 12])
print(f'输入x:\n{x}')
x_batch, y_batch = run_get_batch(x, batch_size=2, context_length=2, device_str='cpu')
print(f'输出x_batch:\n{x_batch}')
print(f'输出y_batch:\n{y_batch}')

In [ ]:
# 6. 保存和加载checkpoint
import torch

def save_checkpoint(model, optimizer, iteration, out):
    """
    将模型、优化器状态和迭代次数保存到目标位置。
    """
    # 创建包含所有必要状态的字典
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'iteration': iteration
    }

    torch.save(checkpoint, out)  # 使用torch.save进行持久化存储

def load_checkpoint(src, model, optimizer):
    """
    从源位置加载检查点，恢复模型和优化器状态。
    返回保存时的迭代次数。
    """

    checkpoint = torch.load(src)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    return checkpoint['iteration']   # 返回迭代次数，以便恢复训练进度

In [ ]:
# 测试save_checkpoint和load_checkpoint函数
import torch
import os
import typing

def run_save_checkpoint(model: torch.nn.Module, optimizer: torch.optim.Optimizer, iteration: int, out: typing.Union[str, os.PathLike, typing.BinaryIO]):

    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'iteration': iteration
    }
    torch.save(checkpoint, out)


def run_load_checkpoint(src: typing.Union[str, os.PathLike, typing.BinaryIO], model: torch.nn.Module, optimizer: torch.optim.Optimizer) -> int:

    # map_location确保了即使是在没有GPU的机器上也能加载GPU保存的模型
    checkpoint = torch.load(src, map_location='cpu')
    
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    return checkpoint['iteration']